Obtaining Acess Token

In [1]:
import requests
import base64

CLIENT_ID = '11bb653f67f44e518bcdd2772b884f96'
CLIENT_SECRET = '054420b840f047708d5d3edaad61c02c'

#we are doing encoding of client id and client secret in base 64
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [3]:
playlist_id = '3bQy66sMaRDIUIsS7UQnuO'
music_df = get_trending_playlist_data(playlist_id, access_token)
print(music_df)

             Track Name                                            Artists   
0        Tumse Hi Tumse                Shekhar Ravjiani, Caralisa Monteiro  \
1   Slow Motion Angreza  Shankar-Ehsaan-Loy, Sukhwinder Singh, Loy Mend...   
2           Lamberghini                               The Doorbeen, Ragini   
3           Matargashti                                      Mohit Chauhan   
4     Kabhi Kabhi Aditi                                         Rashid Ali   
..                  ...                                                ...   
95      Ainvayi Ainvayi  Salim–Sulaiman, Salim Merchant, Sunidhi Chauha...   
96    Gal Mitthi Mitthi                          Amit Trivedi, Tochi Raina   
97            Bang Bang           Vishal-Shekhar, Benny Dayal, Neeti Mohan   
98      Ude Dil Befikre         Vishal-Shekhar, Benny Dayal, Jaideep Sahni   
99        Badtameez Dil               Pritam, Benny Dayal, Shefali Alvares   

                                           Album Name          

In [4]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data=music_df

For building recopmmendation system, we need to give more weight to the latest release (for recommending latest releases)

In [6]:
def calculate_weighted_popularity(release_date):
    release_date=datetime.strptime(release_date,'%Y-%m-%d')
    time_span=datetime.now()-release_date
    weight=1/(time_span.days+1)
    return weight

In [7]:
scaler=MinMaxScaler()
music_features=music_df[['Danceability', 'Energy', 'Key', 
                        'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                        'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled=scaler.fit_transform(music_features)

generating music recommendation system based on music features

In [8]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in our dataset.Pleas enter")
        return
    
    input_song_index=music_df[music_df['Track Name']==input_song_name].index[0]
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]
    return content_based_recommendations

generating music recommendations based on the weighted popularity

In [10]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return
    
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)
    
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]
    
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])
    
    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    new_row = {
    'Track Name': input_song_name,
    'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
    'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
    'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
    'Popularity': weighted_popularity_score
}

# Convert the dictionary to a DataFrame
    new_row_df = pd.DataFrame([new_row])

    # Concatenate the new DataFrame with the existing hybrid_recommendations DataFrame
    hybrid_recommendations = pd.concat([hybrid_recommendations, new_row_df], ignore_index=True)

        
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]
        
    return hybrid_recommendations



In [11]:
input_song_name = "Ude Dil Befikre"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Ude Dil Befikre':
                       Track Name                               Artists   
3                   Badtameez Dil  Pritam, Benny Dayal, Shefali Alvares  \
4                       Sadi Gali                   Lehmber Hussainpuri   
0                Character Dheela   Pritam, Neeraj Shridhar, Amrita Kak   
2                       Suit Suit                  Guru Randhawa, Arjun   
1  Coca Cola (From "Luka Chuppi")  Tony Kakkar, Neha Kakkar, Young Desi   

                Album Name Release Date  Popularity  
3  Yeh Jawaani Hai Deewani   2013-03-30        68.0  
4           Tanu Weds Manu   2011-02-02        64.0  
0                    Ready   2011-04-25        62.0  
2             Hindi Medium   2017-04-21        59.0  
1       Frolic Neha Kakkar   2019-02-28        33.0  
